In [ ]:
# .env 파일에 저장된 환경변수들을 파이썬 환경으로 불러오는 역할
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
# Langchain 프레임워크에서 OpenAI의 ChatOpenAI 모델을 사용하는 예
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
# HuggingFace에서 제공하는 임베딩 모델 "BAAI/bge-m3"를 Langchain을 통해 사용하는 코드
from langchain_huggingface import HuggingFaceEmbeddings
model_name = "BAAI/bge-m3"
hf_embeddings = HuggingFaceEmbeddings(model_name=model_name)

c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Langchain을 사용하여 텍스트 파일을 로드하고, 문서를 청크(조각) 단위로 분리하는 작업
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=0,
    chunk_overlap=0
)

loader = TextLoader("./docs/travel.txt", encoding="utf-8")
documents = loader.load_and_split(text_splitter=text_splitter)

Created a chunk of size 52, which is longer than the specified 0
Created a chunk of size 46, which is longer than the specified 0
Created a chunk of size 43, which is longer than the specified 0
Created a chunk of size 47, which is longer than the specified 0
Created a chunk of size 41, which is longer than the specified 0
Created a chunk of size 41, which is longer than the specified 0
Created a chunk of size 47, which is longer than the specified 0
Created a chunk of size 38, which is longer than the specified 0
Created a chunk of size 40, which is longer than the specified 0
Created a chunk of size 41, which is longer than the specified 0
Created a chunk of size 38, which is longer than the specified 0
Created a chunk of size 46, which is longer than the specified 0
Created a chunk of size 36, which is longer than the specified 0
Created a chunk of size 35, which is longer than the specified 0
Created a chunk of size 45, which is longer than the specified 0
Created a chunk of size 3

In [ ]:
# 문서(documents)를 Chroma 벡터 DB에 저장하기 위한 Langchain 코드
from langchain_chroma import Chroma
db = Chroma.from_documents(documents, hf_embeddings)

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [ ]:
# Chroma 벡터 DB에서 MMR(Maximal Marginal Relevance) 방식을 사용하는 Retriever 객체를 생성
# MMR은 “서로 비슷하지 않으면서도 질문과 관련성 높은 결과” 를 고르는 알고리즘
retriever = db.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 5, "fetch_k": 20, "lambda_mult": 0.5}
)

In [ ]:
# Langchain에서 사용할 **프롬프트 템플릿(ChatPromptTemplate)**을 정의하는 예시로, 
# RAG (Retrieval-Augmented Generation) 흐름에서 매우 일반적으로 사용되는 형태
from langchain_core.prompts import ChatPromptTemplate
message = """
Answer this question using the provided context only.
아래 제공된 내용만 참고하여 사용자의 질문에 답변해줘. 모르면 솔직하게 그냥 모른다고 답해.

Context:
{context}

User Input:
{input}
"""

prompt_template = ChatPromptTemplate.from_messages([
    ("human", message)
])

In [13]:
# LangChain의 Hub 기능을 이용하여 미리 정의된 고품질 프롬프트를 가져오는 예시
from langchain import hub
# https://smith.langchain.com/hub/langchain-ai/retrieval-qa-chat
prompt_template = hub.pull("langchain-ai/retrieval-qa-chat")
prompt_template

c:\Python311\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(input_variables=['context', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMes

In [ ]:
# StrOutputParser는 Langchain에서 출력 파서(Output Parser) 역할을 하는 클래스
# LLM이 생성한 응답을 문자열(str) 형태로 바로 반환하는 가장 단순한 파서
# 별도의 복잡한 파싱(예: JSON 파싱, 키-값 추출 등)이 필요 없고, 단순 텍스트를 그대로 받으려 할 때 사용
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

In [ ]:
# Langchain에서 RAG(검색 증강 생성) 파이프라인을 Runnable 체인으로 구성한 예
from langchain_core.runnables import RunnablePassthrough
rag_chain = {
    "context": retriever,
    "input": RunnablePassthrough()
 } | prompt_template | llm | parser

In [15]:
rag_chain.invoke("이색적인 숙소에는 어떤 곳이 있을까?")

'이색적인 숙소로는 트리하우스, 동굴 호텔, 빙하 호텔 등이 있습니다. 이러한 독특한 숙소 체험이 가능합니다.'

In [16]:
rag_chain.invoke("우주의 역사에 대해서 알려줘")

'주어진 문서 내용에는 우주의 역사에 대한 정보가 포함되어 있지 않습니다. 그러나 일반적으로 우주의 역사는 빅뱅으로 시작하여, 우주가 팽창하고 별과 은하가 형성되며, 생명이 존재하는 행성이 나타나는 과정을 포함합니다. 더 구체적인 내용을 알고 싶다면 다른 출처를 참고하시기를 권장합니다.'